## McMaster University Comp Sci 4TB3/6TB3, Winter Term 2016/17 â Lab 9
#### For the Labs on March 14 - March 16 <br>Due Friday, March 17, 11 pm<br>Emil Sekerinski, Shucai Yao, Spencer Park<br>out of 24 points; extra points count towards any other lab

To be submitted in the folder Lab9 on GitLab; you should have received an invitation to join the project cs4tb3-winter17/yourmacid.
* You need to download the this jupyter notebook and modify it.
* This assignment requires a jupyter installation. You can install it on your computer, use UTS labs, or use https://notebooks.azure.com/.
* In this lab, you are allowed to work in pairs, provided that you split the work equally and arrive at a common understanding of the solution. However, in that case you must state in your submission the person you worked with, such that similarities in the solution will not be construed as Academic Dishonesty. Working in groups of three or larger is not allowed and will be considered Academic Dishonesty. If you look for someone to work with, we will try to find a match, please contact the TAs.
* You are allowed and encouraged to talk to everyone in the course to get a common understanding of the problem, but you can share partial solutions only with your collaborator, if you work in a pair. The final submission must be your own, that is, you cannot submit identical submissions with two names on them.
* In the lab sessions, the solution to last week's lab questions are discussed and you can get help with this week's lab questions. Attendance at the labs is not checked.

### An Interpreter for Functional Programs
This lab is on an interpreter for functional programs with _dynamic binding_, meaning that identifiers are resolved at the time of execution, not compilation. Dynamic binding is used in Python and other interpreted languages. Some examples below illustrate the effect of dynamic binding.

### Lab Question 1 (Completing if-expressions) [12 points]
The implementation below is missing if-expressions. Hence, some of the test cases will fail. Add if-expressions according to the given attribute grammar.

### Lab Question 2 (Dynamic Type Checking) [12 points]
The implementation below lack dynamic type checks. In case there is a type error, a random result is returned. Add type checking according to the examples toward the end.

### Lab Question 3 (Proper Error Positions) [6 points]
The parser reports errors at the proper position in the source text, but the evaluator not. Modify the evaluator accordingly. For this, the nodes in the abstract syntax tree have to be augmented with the position where they occurred in the source text.

### Lab Question 4 (Binary Operator Precedences) [6 points]
Boolean `and`, `or` bind as tight as multiplicative and additive operators. Modify the grammar, the attribute rules, and the parser for following order of decreasing binding power:
- multiplicative operators (`*`, `div`, `mod`)
- additive operators (`+`, `-`)
- relational operators (`=`, `â `, `<`, `>`, `â¤`, `â¥`)
- conjunction (`and`)
- disjunction (`or`)

That is, `a or b * c + d = e and f` is to be parsed in the same way as `a or ((((b * c) + d) = e) and f)`. Do you need to change the evaluator?

#### Scanner

    symbol ::=
        {blank} (identifier | number | 'true' | 'false' | 'let' | 'in' |
        'if' | 'then' | 'else' | 'div' | 'mod' | 'not' | 'and' | 'or' |
        '*' | '+' | '-' | '=' | 'â ' | '<' | '>' | 'â¤' | 'â¥' | '(' | ')' | ',')
    identifier ::= letter {letter | digit}
    number ::= digit {digit}
    letter ::= "a" | ... | "z"
    digit ::= "0" | ... | "9"
    blank ::= " " | "\t" | "\r" | "\n"

Procedure `getChar()` read next character of gobal variable `src` in global variable `ch`; at end, `ch` is set to `chr(0)`. The current position is maintined in the global variable `pos`. Procedure `error(msg)` reports an error as position `pos`.

In [1]:
def getChar():
    global pos, ch
    if pos < len(src): ch, pos = src[pos], pos + 1
    else: ch, pos = chr(0), pos + 1

def error(msg):
    raise Exception(src + '\n' + (pos - 1) * ' ' + '^ ' + msg)

Procedure `getSym()` reads the next symbol in `sym`; at end, `sym` is set to `EOF`; if `sym`
is `NUMBER`, `val` is an integer; if `sym` is `IDENT`, `val` is a string.

In [2]:
IDENT = 0; NUMBER = 1; TRUE = 2; FALSE = 3; LET = 4; IN = 5; IF = 6
THEN = 7; ELSE = 8; DIV = 9; MOD = 10; NOT = 11; AND = 12; OR = 13
TIMES = 14; PLUS = 15; MINUS = 16; EQ = 17; NEQ = 18; LT= 19; GT = 20
LE = 21; GE = 22; LPAREN = 23; RPAREN = 24; COMMA = 25; EOF = 26

KEYWORDS = \
    {'true': TRUE, 'false': FALSE, 'let': LET, 'in': IN, 'if': IF,
    'then': THEN, 'else': ELSE, 'div': DIV, 'mod': MOD, 'not': NOT,
    'and': AND, 'or': OR}

def getSym():
    global sym, val
    while ch in ' \t\r\n': getChar()
    pos0 = pos
    if 'A' <= ch <= 'Z' or 'a' <= ch <= 'z':
        start = pos - 1
        while ('A' <= ch <= 'Z') or ('a' <= ch <= 'z') or \
              ('0' <= ch <= '9'): getChar()
        val = src[start: pos - 1]
        sym = KEYWORDS[val] if val in KEYWORDS else IDENT
    elif '0' <= ch <= '9':
        val = int(ch); getChar()
        while '0' <= ch <= '9':
            val = 10 * val + int(ch); getChar()
        sym = NUMBER
    elif ch == '*': getChar(); sym = TIMES
    elif ch == '+': getChar(); sym = PLUS
    elif ch == '-': getChar(); sym = MINUS
    elif ch == '=': getChar(); sym = EQ
    elif ch == 'â ': getChar(); sym = NEQ
    elif ch == '<': getChar(); sym = LT
    elif ch == '>': getChar(); sym = GT
    elif ch == 'â¤': getChar(); sym = LE
    elif ch == 'â¥': getChar(); sym = GE
    elif ch == '(': getChar(); sym = LPAREN
    elif ch == ')': getChar(); sym = RPAREN
    elif ch == ',': getChar(); sym = COMMA
    elif ch == chr(0): sym = EOF
    else: error('unexpected character')

#### Parser

    expression ::= relation |
        "let" identifier idList = expression "in" expression |
        "if" expression "then" expression "else" expression
    relation ::= arithmetic [("=" | "â " "<" | ">" | "â¤" | "â¥") arithmetic]
    arithmetic ::= ["+" | '-'] term {("+" | "-" | "or") term)}
    term ::= factor {("*" | "div" | "mod" | "and") factor}
    factor ::= integer | "true" | "false" | "(" expression ")" |
        "not" expression | identifier exprList |
    exprList ::= [ "(" expression {"," expression} ")" ]
    idList ::= [ "(" identifier {"," identifier} ")" ]

The abstract syntax tree consists of _nodes_ of following types:

- **`int`** for integer constants
- **`bool`** for boolean constants
- **`Call(func, args)`** with `func` of type `str`, the function name, and list `args` of nodes for the arguments
- **`UnaryExpr(op, arg)`** where `op` is `MINUS` or `NOT` and `arg` is a node for arguments
- **`BinaryExpr(op, left, right)`** where `op` is one of `DIV`, `MOD`, `NOT`, `TIMES`, `PLUS`, `MINUS`, `EQ`, `NEQ`, `LT`, `GT`, `LE`, `GE` and `left`, `right` are nodes for the left and right argument.
- **`If(cond, th, el)`** where `cond`, `th`, `el` are nodes for the condition, then-branch, else-branch
- **`Let(name, par, body, scope)`** where `name` is of type `str` is being defined, parameters `par` is a possibly empty list of `str`, the parameter names, and `body`, `scope` are nodes.


In [3]:
class Call:
    def __init__(self, func, args):
        self.func, self.args = func, args
    def __str__(self):
        return  self.func + \
               '(' + ','.join([str(x) for x in self.args]) + ')'

class Unary:
    def __init__(self, op, operand):
        self.op, self.operand = op, operand
    def __str__(self):
        return 'unary ' + str(self.op) + ' (' + str(self.operand) + ')'

class Binary:
    def __init__(self, op, left, right):
        self.op, self.left, self.right = op, left, right
    def __str__(self):
        return 'binary ' + str(self.op) + ' (' + str(self.left) + \
               ', ' + str(self.right) + ')'

class If:
    def __init__(self, cond, th, el):
        self.cond, self.th, self.el = cond, th, el
    def __str__(self):
        return 'if ' + str(self.cond) + ' then ' + str(self.th) + \
               ' else ' + str(self.el) + ')'

class Let:
    def __init__(self, name, par, body, scope):
        self.name, self.par, self.body, self.scope = \
            name, par, body, scope
    def __str__(self):
        return 'let ' + self.name + '(' + str(self.par) + ') = ' + \
               str(self.body) + ' in ' + str(self.scope)
        
class Node:
    def __init__(self, val, position):
        self.val, self.position = val, position
    def __str__(self):
        return str(self.val)

Procedure `expression()` parses

    expression(e0) ::=
        (relation(e0) |
            "let" identifier(name) idList(par) = expression(body)
                "in" expression(scope) Â«e0 = Let(name, par, body, scope)Â» |
            "if" expression(cond) "then" expression(th) "else" expression(el)
                Â«e0 = Let(cond, th, el)Â»

and returns an abstract syntax tree node or raises an exception with the error.

In [4]:
def expression():
    if sym in (PLUS, MINUS, NUMBER, TRUE, FALSE, LPAREN, NOT, IDENT):
        return relation()
    elif sym == LET:
        getSym()
        if sym == IDENT: getSym()
        else: error("identifier expected")
        name = val; par = idList();
        if sym == EQ: getSym()
        else: error("'=' expected")
        body = expression()
        if sym == IN: getSym()
        else: error("'in' expected")
        scope = expression()
        return Let(name, par, body, scope)
#### Gavin Johnson implementation of "if"
    elif sym == IF:
        getSym()
        cond = expression()
        if sym == THEN: getSym()
        else: error("'then' expected")
        th = expression()
        if sym == ELSE: getSym()
        else: error("'else' expected")
        el = expression()
        return If(cond, th, el)
    else: error("expression expected")

Procedure `relation()` parses

    relation(e0) ::=
        arithmetic(e1)
        ["=" arithmetic(e2) Â«e1 = Binary(EQ, e1, e2)Â» |
            "â " arithmetic(e2) Â«e1 = Binary(NEQ, e1, e2)Â» |
            "<" arithmetic(e2) Â«e1 = Binary(LT, e1, e2)Â» |
            ">" arithmetic(e2) Â«e1 = Binary(GT, e1, e2)Â» |
            "â¤" arithmetic(e2) Â«e1 = Binary(LE, e1, e2)Â» |
            "â¥" arithmetic(e2) Â«e1 = Binary(GE, e1, e2)Â» ]
        Â«e0 = e1Â»

and returns an abstract syntax tree node or raises an exception with the error.

In [5]:
def relation():
    e = arithmetic()
    if sym in (EQ, NEQ, LT, GT, LE, GE):
        op = sym; getSym(); e = Binary(op, e, arithmetic())
    return e

Procedure `arithmetic()` parses

    arithmetic(e0) ::=
        ("+" term(e1) | "-" term(e1) Â«e1 = Unary(MINUS, e1)Â» | term(e1))
        {"+" term(e2) Â«e1 = Binary(PLUS, e1, e2)Â» |
            "-" term(e2) Â«e1 = Binary(MINUS, e1, e2)Â» |
            "or" term(e2) Â«e1 = Binary(OR, e1, e2)Â»}
        e0 = e1

and returns an abstract syntax tree node or raises an exception with the error.

In [6]:
def arithmetic():
    if sym == PLUS:
        getSym(); e = term();
    elif sym == MINUS:
        getSym(); e = Unary(MINUS, term())
    else: e = term()
    while sym in (PLUS, MINUS, OR):
        op = sym; getSym(); e = Binary(op, e, term())
    return e

Procedure `term()` parses

    term(e0) ::=
        factor(e1)
        {"*" factor(e2) Â«e1 = Binary(TIMES, e1, e2)Â» |
            "div" factor(e2) Â«e1 = Binary(DIV, e1, e2)Â» |
            "mod" factor(e2) Â«e1 = Binary(MOD, e1, e2)Â» |
            "and" factor(e2) Â«e1 = Binary(MOD, e1, e2)Â»}
        Â«e0 = e1Â»

and returns an abstract syntax tree node or raises an exception with the error.

In [7]:
def term():
    t = factor()
    while sym in (TIMES, DIV, MOD, AND):
        op = sym; getSym(); t = Binary(op, t, factor())
    return t

Procedure `factor()` parses

    factor(e) ::=
        integer(val) Â«e = valÂ» |
        "true" Â«e = TrueÂ» |
        "false" Â«e = FalseÂ» |
        "(" expression(e) ")" |
        "not" expression(e1) Â«e = not e1Â» |
        identifier(name) exprList(para) Â«e = Call(name, para)Â»

and returns an abstract syntax tree node or raises an exception with the error.

In [46]:
def factor():
    p = pos
    if sym == NUMBER: e = val; p = pos; getSym()
    elif sym == TRUE: e = True; p = pos; getSym()
    elif sym == FALSE: e = False; p = pos; getSym()
    elif sym == LPAREN:
        getSym(); e = expression()
        if sym == RPAREN: getSym()
        else: error(') missing')
    elif sym == NOT:
        getSym(); e = Unary(NOT, expression())
    elif sym == IDENT:
        p = pos
        name = val; getSym()
        para = exprList()
        e = Call(name, para)
    else: error('unexpected symbol')
    return Node(e, p-1)


    exprList(el) ::=
        ("(" expression(e) Â«el = [e]Â» {"," expression(e) Â«el.append(e)Â»} ")") |
        Â«el = []Â»

In [47]:
def exprList():
    if sym == LPAREN:
        getSym(); el = [expression()]
        while sym == COMMA:
            getSym(); el.append(expression())
        if sym == RPAREN: getSym()
        else: error(') expected')
    else: el = []
    return el

    idList(il) ::=
        ("(" identifer(i) Â«il = [i]Â» {"," identifier(i) Â«il.append(i)Â»} ")") |
        Â«il = []Â»

In [48]:
def idList():
    if sym == LPAREN:
        getSym()
        if sym == IDENT: il = [val]; getSym()
        else: error('identifier expected')
        while sym == COMMA:
            getSym();
            if sym == IDENT: il.append(val); getSym()
            else: error('identifier expected')
        if sym == RPAREN: getSym()
        else: error(') expected')
    else: il = []
    return il

In [49]:
def ast(s):
    global src, pos;
    src, pos = s, 0; getChar(); getSym();
    return expression()

####  Abstract Syntax Tree Tests

Following tests should return `True`.

In [50]:
def testast(s):
    try: return str(ast(s))
    except Exception as e:
        print(e)

In [51]:
testast('(a)') == 'a()'

True

In [52]:
testast('-a') == 'unary 16 (a())'

True

In [53]:
testast('a+2') == 'binary 15 (a(), 2)'

True

In [54]:
testast('a+2 *  c') == 'binary 15 (a(), binary 14 (2, c()))'

True

In [55]:
testast('f(a,b+c) div d') == 'binary 9 (f(a(),binary 15 (b(), c())), d())'

True

In [56]:
testast('(a + b) * (c mod d)') == 'binary 14 (binary 15 (a(), b()), binary 10 (c(), d()))'

True

In [57]:
testast('-a-b') == 'binary 16 (unary 16 (a()), b())'

True

In [58]:
testast('if x = 3 then b + x else d') == 'if binary 17 (x(), 3) then binary 15 (b(), x()) else d())'

True

In [59]:
testast('f(3, 4)') == 'f(3,4)'

True

In [60]:
testast('let f(a, b) = a + b in f(3, 4)') == "let f(['a', 'b']) = binary 15 (a(), b()) in f(3,4)"

True

Following tests should produce an error message.

Error "unexpected character"

In [61]:
testast('a $')

a $
  ^ unexpected character


Error "unexpected symbol"

In [62]:
testast('-a * -b')

-a * -b
      ^ unexpected symbol


Error "unexpected symbol"

In [63]:
testast('a + *') # error

a + *
     ^ unexpected symbol


Error "unexpected symbol"

In [64]:
testast('a + ') # error

a + 
    ^ unexpected symbol


Error ") missing"

In [65]:
testast('(a+b') # error

(a+b
    ^ ) missing


#### Evaluator

The interpreter is realized by the function `eval(e, env)`, which evaluates expression `e` in environment `env`. An environment is a dictionary with identifiers as keys. A value is either a pair with function parameter and function body, or a constant. For example, the call

    eval('let f(x) = x + x in f(3)', {})

evaluates `f(3)` in the environment:

    {'f': (['x'], ast('x + x'))}`

The evaluation of `f(3)` leads further to the evaluation of `x + x` in the environment:

    {'f': (['x'], ast('x + x')), 'x': 3}

Function `eval(e, env)` is defined recursively over the structure of expressions.

In [116]:
def eval(e, env):
#### Gavin Johnson implementation of type-checking
    global pos
    if type(e) in (int, bool): return e
    #### Gavin Johnson evaluation position tracking
    if type(e) == Node:
        pos = e.position
        return eval(e.val, env)
    elif type(e) == Unary:
        if e.op == MINUS:
            f = eval(e.operand, env)
            if type(f) == int:  return - f
            else: error("operand not integer")
        elif e.op == NOT:
            f = eval(e.operand, env)
            if type(f) == bool: return not f
            else: error("operand not boolean")
        else: assert False
    elif type(e) == Binary:
        if e.op in (DIV, MOD, TIMES, PLUS, MINUS, LT, GT, LE, GE):
            l = eval(e.left, env)
            if type(l) != int: error("incompatible operands")
            r = eval(e.right, env)
            if type(r) != int: error("incompatible operands")
            if e.op == DIV: return l // r
            elif e.op == MOD: return l % r
            elif e.op == TIMES: return l * r
            elif e.op == PLUS: return l + r
            elif e.op == MINUS: return l - r
            elif e.op == LT: return l < r
            elif e.op == GT: return l > r
            elif e.op == LE: return l <= r
            elif e.op == GE: return l >= r
        elif e.op in (AND, OR):
            l = eval(e.left, env)
            if type(l) != bool: error("incompatible operands")
            r = eval(e.right, env)
            if type(r) != bool: error("incompatible operands")
            if e.op == AND: return l and r
            elif e.op == OR: return l or r
        elif e.op in (EQ, NEQ):
            l, r = eval(e.left, env), eval(e.right, env)
            if type(l) == type(r) and type(l) in {int, bool}:
                if e.op == EQ: return l == r
                elif e.op == NEQ: return l != r
            else: error("incompatible operands")
#### Gavin Johnson implementation of "if"
    elif type(e) == If:
        cond = eval(e.cond, env)
        if type(cond) == bool:
            if cond:
                return eval(e.th, env)
            else: return eval(e.el, env)
        else: error("condition not boolean")
    elif type(e) == Call:
        if e.func in env:
            if type(env[e.func]) in {int, bool}:
                return env[e.func]
            else: # must be call
                par, body = env[e.func]
                if len(e.args) == len(par):
                    s =env.copy()
                    s.update(zip(par, [eval(a, env) for a in e.args]))
                    return eval(body, s)
                else: error("number of parameters doesn't match")
        else: error('identifier not defined')
    elif type(e) == Let:
        s = env.copy(); s.update({e.name: (e.par, e.body)})
        return eval(e.scope, s)
    else: assert False

In [117]:
def evaluate(s):
    try:
        return eval(ast(s), {})
    except Exception as e:
        print(e)

#### Tests
Following tests should return `True`.

In [118]:
evaluate('4 + 2 * 3 mod 5') == 5

True

In [119]:
evaluate('let x = 3 in x + x') == 6

True

In [120]:
evaluate('if 2 < 5 then 7 else 8') == 7

True

In [121]:
evaluate('let f(p) = p + p in f(3)') == 6

True

In [122]:
evaluate('let x = 3 > 5 in if x or (3 < 5) then x else not x') == False

True

In [123]:
evaluate('let mult(x, y) = if y = 0 then 0 else x + mult(x, y - 1) in mult(2, 3)') == 6

True

In [124]:
evaluate("""
let gcd(x, y) = if y â  0 then gcd(y, x mod y) else x in gcd(25, 15)""") == 5

True

Following example illustrate dynamic binding: the expression when the expression `f(4` is evaluated, the environment has definitions of both `f` and `x`, hence the programs are well-defined. In case there are multiple definitions, the last dynamic definition is taken

In [125]:
evaluate('let x = 7 in let f(p) = p - x in f(4)') == -3

True

In [126]:
evaluate('let f(p) = p - x in let x = 7 in f(4)') == -3

True

In [127]:
evaluate('let x = 1 in let f(p) = p - x in let x = 7 in f(4)') == -3

True

Following tests should procedure an error message.

Error "identifier not defined"

In [128]:
evaluate('let f(p) = x + 3 in f(2)')

let f(p) = x + 3 in f(2)
           ^ identifier not defined


Error "incompatible operands": arithmetic operators must have integer operands, boolean operators must have boolean operands, = and â  must have both operands integer or boolean.

In [129]:
evaluate('3 + true')

3 + true
       ^ incompatible operands


Error "operand not integer": "-" must have integer operand.

In [130]:
evaluate('- true')

- true
     ^ operand not integer


Error "operand not boolean": "not" must have boolean operand

In [131]:
evaluate('not 4')

not 4
    ^ operand not boolean


Error "condition not boolean": condition of "if" must be boolean

In [132]:
evaluate('if 4 then 3 else 7')

if 4 then 3 else 7
   ^ condition not boolean


Error "number of parameters doesn't match": definition and use of a function must have matching number of parameters; constants are functions without parameters.

In [133]:
evaluate('let f = 4 in f(5)')

let f = 4 in f(5)
             ^ number of parameters doesn't match


In [134]:
evaluate('((true and true) or (false and false)) + (4)')

((true and true) or (false and false)) + (4)
                                   ^ incompatible operands


#### Appendix

In [364]:
%%HTML
<style>
div.prompt {display:none}
</style>